### Examples
```python
class Item(BaseModel): 
  name: str = Field(examples=["Foo"])
  price: float = Field(examples=[35.4])
```

In [1]:
from fastapi import FastAPI

app = FastAPI()

### path + body + query
- Если параметр также указан в пути, то он будет использоваться как параметр пути.
- Если аннотация типа параметра содержит примитивный тип (`int`, `float`, `str`, `bool` и т.п.), он будет интерпретирован как параметр запроса.
- Если аннотация типа параметра представляет собой модель Pydantic, он будет интерпретирован как параметр тела запроса.

- [Path()](#path-params)
- [Query()](#query-params)
- [Header()]()
- [Cookie()]()
- [Body()](#body-params)
- [Form()]()
- [File()]()

### Path params
- Можно использовать [enum](https://fastapi.tiangolo.com/ru/tutorial/path-params/#predefined-values)
- При указании типа производится:
  1. [парсинг](https://fastapi.tiangolo.com/ru/tutorial/path-params/#_1)
  2. [проверка типа](https://fastapi.tiangolo.com/ru/tutorial/path-params/#_3)
  3. [auto-doc](https://fastapi.tiangolo.com/tutorial/metadata/)


In [ ]:
# http://127.0.0.1:8000/my-param
@app.get('/items/{item_id}')     # 1. {}
def read_item(item_id: int):     # 2. pass into function
    return {'item_id': item_id}  # 3. get in page

### Query params
- Смешивать `Query` и `Path`-параметры можно в любом порядке
- Можно использовать [enum](https://fastapi.tiangolo.com/ru/tutorial/path-params/#predefined-values)

In [ ]:
from typing import Annotated
from fastapi import Query

In [ ]:
@app.get('/')

# query-параметры по умолчанию
def get_home(sort: str = 'asc', limit: int = 10):
    return {'Hello': 'World', 'sort': sort, 'limit': limit}


# http://127.0.0.1:8000/ == http://127.0.0.1:8000/?sort=asc&limit=10


# Обязательные параметры
def get_home(reqParam: bool):
    pass
def get_home(reqParam: bool = ...):
    pass


# Обязательные, но со значением по умолчанию
def get_home(someParam: bool = False):
    pass


# Необязательные параметры (None по умолчанию)
def get_home(showTables: str | None = None):
    pass


# List of query params
def get_home(q: Annotated[list[str], Query()]):
    pass  # ?q=1&q=2 -> {"q": ["1", "2"]}


# Default list of query params
def get_home(q: Annotated[list[str], Query()] = ['1', '2']):
    pass


# Default with None
def get_home(q: Annotated[str | None, Query()] = ...):
    pass

### Body params
наследуются от [pandentic.BaseModel](https://docs.pydantic.dev/latest/concepts/models/)

In [ ]:
from pydantic import BaseModel

class Item(BaseModel):              # json, парсинг, валидация, генерация docs
    name: str                       # Обязательное поле модели
    description: str | None = None  # НЕобязательное поле
    price: float
    tax: float | None = None

@app.post('/')
def create_item(item: Item):
    return item

In [ ]:
from fastapi import Body

@app.post('/body/')
def do_something(param: str = Body()) -> str:
    return f'Your param is: {param}'

In [ ]:
@app.post('/body/')  
async def body_foo(body: str):  
    return body

### Bool params

In [ ]:
def get_home(showInfo: bool = False):
    pass

# В http: (True, true, 1, on, yes)
# http://127.0.0.1:8000/?showInfo=yes

### Predefined params
Используется библиотека [enum](../../04_builtin/enum.ipynb)

In [ ]:
from enum import Enum

class ParamType(str, Enum):
    a = 'value1'
    b = 'value2'
    c = 'value3'

app = FastAPI()

@app.get('/{param}')
def read_root(param: ParamType):
    if param is ParamType.a:  # Можно сравнивать так
        return {'param': ParamType.a}
    if param.value == 'value2':  # Или так
        return {'param': param.value}  # Получение значения из Enum
    return {'param': param}